In [1]:
import os
import sys

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

from pyspark import SparkContext, SparkConf

#We can create a SparkConf() object and use it to initialize the spark context
conf = SparkConf().setAppName("Notebook 1").setMaster("local[4]") #Initialize spark context using 4 local cores as workers
sc = SparkContext(conf=conf)    

from pyspark.rdd import RDD

23/11/18 16:32:28 WARN Utils: Your hostname, abdou-Nitro-AN515-55 resolves to a loopback address: 127.0.1.1; using 10.188.49.204 instead (on interface wlp0s20f3)
23/11/18 16:32:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/18 16:32:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Words Count on the Moby Dick text

![MobyDick](https://d28hgpri8am2if.cloudfront.net/book_images/onix/cvr9781681778488/moby-dick-9781681778488_xlg.jpg)

And so begins an incredible odyssey that will take ship, sailors and readers to the edge of darkness. Moby Dick" is an epic tragedy of terrible dramatic power, the endless, desperate quest of the captain of a drunken ship on a voyage of no return.

In [7]:
%%time
text_file = sc.textFile("../data/mobydick.txt")
type(text_file)

CPU times: user 0 ns, sys: 1.75 ms, total: 1.75 ms
Wall time: 45.6 ms


pyspark.rdd.RDD

In [8]:
%%time
words = text_file.flatMap(lambda line: line.split(" "))

CPU times: user 1.26 ms, sys: 103 µs, total: 1.37 ms
Wall time: 931 µs


In [9]:
%%time
not_empty = words.filter(lambda x: x!="")

CPU times: user 28 µs, sys: 2 µs, total: 30 µs
Wall time: 33.4 µs


In [10]:
%%time
key_val = not_empty.map(lambda x: (x, 1))
counts= key_val.reduceByKey(lambda x1, x2: x1+x2)

CPU times: user 7.72 ms, sys: 0 ns, total: 7.72 ms
Wall time: 50.8 ms


In [14]:
%%time
## get the number of different words
diff_words = counts.count()
## get the number of words
nb_words = counts.map(lambda x: x[1]).reduce(lambda x, y: x+y)

print(f"number of different words = {diff_words} number of words = {nb_words} nb of occurence per word = {round(nb_words / diff_words, 2)}")

number of different words = 19840 number of words = 115314 nb of occurence per word = 5.81
CPU times: user 7.52 ms, sys: 0 ns, total: 7.52 ms
Wall time: 205 ms


#### To sum up

- An RDD is a distributed immutable array and is the core data structure of Spark
- It is not possible to operate on RDD directly but through **Transformations** and **Actions**
- **Transformations** transform an RDD into another RDD
- **Actions** output their results on the head node
- After the action is done you are now using the head node and no longer on the workers node.
- RDD operations (**Transformations** and **Actions**) are added to what we call an **Execution Plan**
- The plan is executed when the result is needed
- It is possible to store intermediate result explicitly by using caching